In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import datetime
import ast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,TimeDistributed,Activation,Reshape,Flatten,GRU,Conv2D,MaxPooling2D,BatchNormalization,LSTM,Reshape
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
dataset=pd.read_csv('C:/Users/Lenovo/AutoCuroAssignment/tic_tac_toe_records_minimax_Preprocessed_DNN.csv')

In [13]:
dataset

,board,decision,Input
0,OX O XX,3,"[2, 1, 0, 0, 0, 2, 0, 1, 1]"
1,XX XXOOO,9,"[1, 1, 0, 1, 1, 2, 2, 2, 0]"
2,X O,4,"[0, 1, 0, 0, 2, 0, 0, 0, 0]"
3,O X OX X,5,"[2, 0, 1, 0, 0, 2, 1, 0, 1]"
4,XOXO OX,9,"[1, 2, 1, 2, 0, 0, 2, 1, 0]"
...,...,...,...
10015,,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
10016,XXOXXO O,9,"[1, 1, 2, 1, 1, 2, 0, 2, 0]"
10017,XO XOX,7,"[1, 2, 0, 1, 2, 1, 0, 0, 0]"
10018,XX O XO,3,"[1, 1, 0, 0, 0, 2, 0, 1, 2]"


In [14]:
dataset['Input'] = dataset['Input'].apply(ast.literal_eval)
X=dataset['Input']
y=dataset['decision']

In [15]:
y=y-1

In [16]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
print((type(X_train),
type(y_train)))

(<class 'pandas.core.series.Series'>, <class 'pandas.core.series.Series'>)


In [19]:
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [22]:
X_test.shape

(2004, 9)

In [23]:
model = Sequential()

model.add(LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(LSTM(128, return_sequences=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(9, activation='softmax'))

In [24]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set up callbacks
checkpoint = ModelCheckpoint(
    'best_model.h5',  
    monitor='val_loss',  
    save_best_only=True,  
    mode='min', 
    verbose=1 
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True 
)


history = model.fit(
    X_train, y_train, 
    epochs=200, 
    batch_size=32,  
    validation_data=(X_test, y_test),  
    callbacks=[checkpoint, early_stopping]  
)

Epoch 1/200
250/251 [============================>.] - ETA: 0s - loss: 2.1713 - accuracy: 0.2149
Epoch 1: val_loss improved from inf to 2.14532, saving model to best_model.h5
251/251 [==============================] - 11s 16ms/step - loss: 2.1704 - accuracy: 0.2154 - val_loss: 2.1453 - val_accuracy: 0.2201
Epoch 2/200
251/251 [==============================] - ETA: 0s - loss: 1.9857 - accuracy: 0.2781
Epoch 2: val_loss improved from 2.14532 to 2.04824, saving model to best_model.h5
251/251 [==============================] - 3s 11ms/step - loss: 1.9857 - accuracy: 0.2781 - val_loss: 2.0482 - val_accuracy: 0.2410
Epoch 3/200
247/251 [============================>.] - ETA: 0s - loss: 1.8949 - accuracy: 0.3035
Epoch 3: val_loss improved from 2.04824 to 1.79758, saving model to best_model.h5
251/251 [==============================] - 3s 11ms/step - loss: 1.8947 - accuracy: 0.3035 - val_loss: 1.7976 - val_accuracy: 0.3253
Epoch 4/200
247/251 [============================>.] - ETA: 0s - loss:

In [25]:
from tensorflow.keras.models import load_model
model = load_model('C:/Users/Lenovo/AutoCuroAssignment/best_model.h5')

In [26]:
y_pred=model.predict(X_test)

63/63 [==============================] - 1s 4ms/step


In [27]:
y_pred

array([[8.2798081e-04, 9.7515248e-03, 3.8760909e-01, ..., 3.5219942e-03,
        8.1960998e-06, 1.5075077e-05],
       [2.7257384e-04, 2.6797945e-06, 1.7829563e-03, ..., 2.9332255e-04,
        5.4564855e-05, 9.5071355e-09],
       [3.1467743e-02, 7.5092740e-02, 1.1400519e-06, ..., 1.3123500e-01,
        4.3613079e-05, 1.3698797e-01],
       ...,
       [9.7298570e-02, 1.1493265e-01, 1.0948007e-01, ..., 1.1562131e-01,
        1.0743031e-01, 1.0253124e-01],
       [2.7257437e-04, 2.6797945e-06, 1.7829554e-03, ..., 2.9332284e-04,
        5.4564909e-05, 9.5071355e-09],
       [2.7257437e-04, 2.6797945e-06, 1.7829554e-03, ..., 2.9332284e-04,
        5.4564909e-05, 9.5071355e-09]], dtype=float32)

In [28]:
y_pred = [ np.argmax(label) for label in y_pred]
y_pred

[4,
 4,
 4,
 2,
 4,
 5,
 5,
 8,
 5,
 0,
 0,
 2,
 4,
 4,
 4,
 4,
 1,
 7,
 0,
 8,
 0,
 2,
 1,
 7,
 1,
 4,
 5,
 4,
 4,
 4,
 2,
 5,
 2,
 0,
 5,
 2,
 8,
 0,
 8,
 2,
 4,
 0,
 7,
 2,
 5,
 5,
 6,
 5,
 2,
 6,
 4,
 7,
 1,
 0,
 4,
 2,
 6,
 8,
 8,
 5,
 5,
 4,
 4,
 8,
 4,
 4,
 4,
 8,
 7,
 2,
 6,
 3,
 4,
 4,
 3,
 7,
 5,
 8,
 8,
 4,
 0,
 7,
 4,
 6,
 1,
 8,
 5,
 2,
 8,
 0,
 8,
 5,
 2,
 5,
 5,
 1,
 2,
 5,
 4,
 4,
 4,
 6,
 6,
 3,
 0,
 2,
 7,
 4,
 0,
 3,
 4,
 8,
 8,
 3,
 8,
 8,
 3,
 2,
 5,
 3,
 6,
 5,
 2,
 5,
 5,
 4,
 4,
 8,
 2,
 8,
 1,
 0,
 4,
 5,
 4,
 4,
 0,
 0,
 1,
 2,
 0,
 2,
 7,
 0,
 5,
 4,
 6,
 8,
 5,
 2,
 0,
 6,
 6,
 0,
 2,
 8,
 2,
 6,
 4,
 8,
 0,
 5,
 5,
 6,
 0,
 4,
 2,
 4,
 8,
 4,
 5,
 2,
 0,
 4,
 4,
 1,
 4,
 4,
 8,
 0,
 6,
 7,
 6,
 5,
 4,
 5,
 6,
 2,
 5,
 4,
 5,
 0,
 5,
 5,
 2,
 1,
 2,
 4,
 6,
 4,
 7,
 0,
 4,
 8,
 6,
 4,
 1,
 2,
 0,
 4,
 2,
 0,
 4,
 5,
 4,
 5,
 2,
 8,
 2,
 1,
 5,
 7,
 0,
 5,
 7,
 2,
 4,
 0,
 4,
 0,
 8,
 5,
 4,
 4,
 3,
 3,
 8,
 4,
 5,
 7,
 4,
 8,
 2,
 2,
 4,
 5,
 2,
 8,
 6,
 0,


In [29]:
from sklearn.metrics import classification_report,confusion_matrix
class_labels=['1','2','3','4','5','6','7','8','9']
cr=classification_report(y_test,y_pred,target_names=class_labels)
cm=confusion_matrix(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           1       0.59      0.49      0.54       227
           2       0.50      0.45      0.48       133
           3       0.53      0.63      0.58       245
           4       0.48      0.35      0.40       121
           5       0.78      0.76      0.77       441
           6       0.27      0.68      0.39       152
           7       0.66      0.43      0.52       273
           8       0.65      0.43      0.52       150
           9       0.64      0.59      0.62       262

    accuracy                           0.57      2004
   macro avg       0.57      0.53      0.53      2004
weighted avg       0.61      0.57      0.58      2004



In [26]:
#test output
import numpy as np
import pandas as pd
from keras.models import load_model
model=load_model('C:/Users/Lenovo/AutoCuroAssignment/best_model.h5')
data_input=[1,1,0,2,2,0,1,1,0]
data_array=np.array(data_input)
data_array = np.array(data_array.tolist())
data_array = data_array.astype(np.float32)
data_array=data_array.reshape(1, 9)

In [29]:
data_array.shape

(1, 9)

In [30]:
print(np.argmax(model.predict(data_array)))

1/1 [==============================] - 1s 1s/step
2
